In [11]:
import cassandra;

print(cassandra.__version__)

3.24.0


In [12]:
def print_result_set(result_set):
    for row in result_set:
        print(row)

In [13]:
from cassandra.cluster import Cluster

cluster = Cluster()
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS items
    WITH replication = {
      'class' : 'SimpleStrategy',
      'replication_factor' : 1
    };
""")
session.set_keyspace("items")

In [14]:
session.execute("""
        CREATE TABLE IF NOT EXISTS products (
        category text,
        price int,
        producer text,
        name text,
        characteristics map<text, text>,
        PRIMARY KEY (category, price, producer)
        )
    """)


# indexes

# session.execute("""
#         CREATE MATERIALIZED VIEW products_by_name 
#         AS SELECT category, price, producer, name, characteristics 
#         FROM products 
#         PRIMARY KEY (name);
#     """)
# looks like materialized view is not recommended to use
session.execute("""
        CREATE INDEX IF NOT EXISTS products_name_index
        ON products (name)
    """)

session.execute("""
        CREATE INDEX IF NOT EXISTS product_characteristics_idx
        ON products ( KEYS (characteristics) );
    """)

session.execute("""
        CREATE INDEX IF NOT EXISTS product_characteristics_entries_idx
        ON products ( ENTRIES(characteristics) );
    """)


In [15]:
session.execute("""
        INSERT INTO products (category, name, price, producer, characteristics) 
        VALUES ('phone', 'iPhone4', 600, 'Apple', {'cpu': '4', 'diagonal': '5'})
    """)
session.execute("""
        INSERT INTO products (category, name, price, producer, characteristics) 
        VALUES ('phone', 'iPhone5', 800, 'Apple', {'cpu': '6', 'diagonal': '6'})
    """)
session.execute("""
        INSERT INTO products (category, name, price, producer, characteristics) 
        VALUES ('tv', 'SuperTV', 1000, 'Samsung', {'diagonal': '60', '4k': 'True'})
    """)
session.execute("""
        INSERT INTO products (category, name, price, producer, characteristics) 
        VALUES ('watch', 'iWatch', 300, 'Apple', {'wi-fi': 'True', 'heartsensors': 'True'})
    """)
session.execute("""
        INSERT INTO products (category, name, price, producer, characteristics) 
        VALUES ('keyboard', 'MX', 50, 'Logitech', {'cpu': '4', 'diagonal': '6'})
    """)
session.execute("""
        INSERT INTO products (category, name, price, producer, characteristics) 
        VALUES ('phone', 'iPhone6', 1000, 'Apple', {'cpu': '8', 'diagonal': '6'})
    """)

In [16]:
# Напишіть запит, який показує структуру створеної таблиці
print_result_set(session.execute("""DESCRIBE TABLE products"""))

Row(keyspace_name='items', type='table', name='products', create_statement="CREATE TABLE items.products (\n    category text,\n    price int,\n    producer text,\n    name text,\n    characteristics map<text, text>,\n    PRIMARY KEY (category, price, producer)\n) WITH CLUSTERING ORDER BY (price ASC, producer ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    A

In [17]:
# Напишіть запит, який виводить усі товари в певній категорії відсортовані за ціною
print_result_set(
    session.execute("""
        SELECT * FROM products WHERE category = 'phone'
    """)
)

Row(category='phone', price=600, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '4'), ('diagonal', '5')]), name='iPhone4')
Row(category='phone', price=800, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '6'), ('diagonal', '6')]), name='iPhone5')
Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '8'), ('diagonal', '6')]), name='iPhone6')


In [58]:
# Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії:
# назва
print_result_set(
    session.execute("""
        SELECT * FROM products WHERE category = 'phone' AND name = 'iPhone5'
    """)
)

Row(category='phone', price=800, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '6'), ('diagonal', '6')]), name='iPhone5')


In [60]:
# ціна (в проміжку)
print_result_set(
    session.execute("""
        SELECT * FROM products WHERE category = 'phone' AND price < 1000 AND price > 600
    """)
)

Row(category='phone', price=800, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '6'), ('diagonal', '6')]), name='iPhone5')


In [63]:
# ціна та виробник 
print_result_set(
    session.execute("""
        SELECT * FROM products WHERE category = 'phone' AND price=1000 AND producer = 'Apple'
    """)
)

Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '8'), ('diagonal', '6')]), name='iPhone6')


In [67]:
# Напишіть запити, які вибирають товари за:
# наявність певних характеристик
print_result_set(
    session.execute("""
        SELECT * FROM products WHERE characteristics CONTAINS KEY 'cpu';
    """)
)

Row(category='phone', price=600, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '4'), ('diagonal', '5')]), name='iPhone4')
Row(category='phone', price=800, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '6'), ('diagonal', '6')]), name='iPhone5')
Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '8'), ('diagonal', '6')]), name='iPhone6')
Row(category='keyboard', price=50, producer='Logitech', characteristics=OrderedMapSerializedKey([('cpu', '4'), ('diagonal', '6')]), name='MX')


In [81]:
# певна характеристика та її значення
print_result_set(
    session.execute("""
        SELECT * FROM products WHERE characteristics['cpu']='8';
    """)
)

Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '8'), ('diagonal', '6')]), name='iPhone6')


In [88]:
# Оновити опис товару:
# змінить існуючі значення певної характеристики 
print_result_set(
    session.execute("""
        UPDATE products
        SET characteristics['cpu'] = '12'
        WHERE category='phone' AND price=1000 AND producer='Apple';
    """)
)

In [89]:
print_result_set(
    session.execute("""
        SELECT * FROM products
        WHERE category='phone' AND price=1000 AND producer='Apple';
    """)
)

Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '12'), ('diagonal', '6')]), name='iPhone6')


In [90]:
# додайте нові властивості (характеристики) товару
print_result_set(
    session.execute("""
        UPDATE products
        SET characteristics = characteristics + {'expensive': 'True'}
        WHERE category='phone' AND price=1000 AND producer='Apple';
    """)
)
print_result_set(
    session.execute("""
        SELECT * FROM products
        WHERE category='phone' AND price=1000 AND producer='Apple';
    """)
)

Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '12'), ('diagonal', '6'), ('expensive', 'True')]), name='iPhone6')


In [91]:
# видалить характеристику товару
print_result_set(
    session.execute("""
        DELETE characteristics['expensive']
        FROM products
        WHERE category='phone' AND price=1000 AND producer='Apple';
    """)
)
print_result_set(
    session.execute("""
        SELECT * FROM products
        WHERE category='phone' AND price=1000 AND producer='Apple';
    """)
)

Row(category='phone', price=1000, producer='Apple', characteristics=OrderedMapSerializedKey([('cpu', '12'), ('diagonal', '6')]), name='iPhone6')


**Створіть таблицю orders в якій міститься ім'я замовника і інформація про замовлення: перелік id-товарів у замовленні, вартість замовлення, дата замовлення, .... Для кожного замовника повинна бути можливість швидко шукати його замовлення і виконувати по них запити.**

In [93]:
session.execute("""
        CREATE TABLE IF NOT EXISTS orders (
        client_name text,
        date timestamp,
        products list<int>,
        id int,
        PRIMARY KEY (client_name, date, id)
        )
    """)
session.execute("""
        CREATE TABLE IF NOT EXISTS order_price (
        price counter,
        id int, 
        PRIMARY KEY (id)
        )
    """)


session.execute("""
        INSERT INTO orders (client_name, date, products, id) 
        VALUES ('Ivan', '2020-12-12T12:00:00', [1, 2, 3], 0)
    """)
session.execute("""
        UPDATE order_price
        SET price = price + 600
        WHERE id = 0
    """)

session.execute("""
        INSERT INTO orders (client_name, date, products, id) 
        VALUES ('Ivan', '2019-12-12T12:00:00', [5], 1)
    """)
session.execute("""
        UPDATE order_price
        SET price = price + 300
        WHERE id = 1
    """)

session.execute("""
        INSERT INTO orders (client_name, date, products, id) 
        VALUES ('Ivan', '2017-12-12T12:00:00', [4], 2)
    """)
session.execute("""
        UPDATE order_price
        SET price = price + 50
        WHERE id = 2
    """)

session.execute("""
        INSERT INTO orders (client_name, date, products, id) 
        VALUES ('Victor', '2018-10-12T12:00:00', [1], 3)
    """)
session.execute("""
        UPDATE order_price
        SET price = price + 200
        WHERE id = 3
    """)

session.execute("""
        INSERT INTO orders (client_name, date, products, id) 
        VALUES ('Victor', '2019-10-12T12:00:00', [10, 11], 4)
    """)
session.execute("""
        UPDATE order_price
        SET price = price + 2000
        WHERE id = 4
    """)

session.execute("""
        INSERT INTO orders (client_name, date, products, id) 
        VALUES ('Fedya', '2020-10-01T12:00:00', [1, 2, 10], 5)
    """)
session.execute("""
        UPDATE order_price
        SET price = price + 1400
        WHERE id = 5
    """)


In [94]:
session.execute("""
        CREATE INDEX IF NOT EXISTS products_id_index
        ON orders (products)
    """)

In [95]:
print_result_set(session.execute("""select * from orders"""))

Row(client_name='Ivan', date=datetime.datetime(2017, 12, 12, 12, 0), id=2, products=[4])
Row(client_name='Ivan', date=datetime.datetime(2019, 12, 12, 12, 0), id=1, products=[5])
Row(client_name='Ivan', date=datetime.datetime(2020, 12, 12, 12, 0), id=0, products=[1, 2, 3])
Row(client_name='Fedya', date=datetime.datetime(2020, 10, 1, 12, 0), id=5, products=[1, 2, 10])
Row(client_name='Victor', date=datetime.datetime(2018, 10, 12, 12, 0), id=3, products=[1])
Row(client_name='Victor', date=datetime.datetime(2019, 10, 12, 12, 0), id=4, products=[10, 11])


In [96]:
# Напишіть запит, який показує структуру створеної таблиці (команда DESCRIBE) 
print_result_set(session.execute("""DESCRIBE TABLE orders"""))

Row(keyspace_name='items', type='table', name='orders', create_statement="CREATE TABLE items.orders (\n    client_name text,\n    date timestamp,\n    id int,\n    products list<int>,\n    PRIMARY KEY (client_name, date, id)\n) WITH CLUSTERING ORDER BY (date ASC, id ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND specul

In [98]:
# Для замовника виведіть всі його замовлення відсортовані за часом коли вони були зроблені
print_result_set(session.execute("""select * from orders where client_name='Ivan'"""))

Row(client_name='Ivan', date=datetime.datetime(2017, 12, 12, 12, 0), id=2, products=[4])
Row(client_name='Ivan', date=datetime.datetime(2019, 12, 12, 12, 0), id=1, products=[5])
Row(client_name='Ivan', date=datetime.datetime(2020, 12, 12, 12, 0), id=0, products=[1, 2, 3])


In [99]:
# Для замовника знайдіть замовлення з певним товаром
print_result_set(session.execute("""select * from orders where client_name='Ivan' AND products CONTAINS 1"""))

Row(client_name='Ivan', date=datetime.datetime(2020, 12, 12, 12, 0), id=0, products=[1, 2, 3])


In [100]:
# Для замовника знайдіть замовлення за певний період часу і їх кількість
print_result_set(session.execute("""
    select * from orders where client_name='Ivan' AND date < '2020-12-11T12:00:00' AND date > '2017-12-25T12:00:00'
"""))
print_result_set(session.execute("""
    select count(*) from orders where client_name='Ivan' AND date < '2020-12-11T12:00:00' AND date > '2017-12-25T12:00:00'
"""))


Row(client_name='Ivan', date=datetime.datetime(2019, 12, 12, 12, 0), id=1, products=[5])
Row(count=1)


In [127]:
# Для кожного замовників визначте середню вартість замовлення
ivan_orders_ids = [str(row.id) for row in session.execute("""
    select id from orders where client_name='Ivan'
""")]
print_result_set(
    session.execute("select avg(price) from order_price where id in (" + ', '.join(ivan_orders_ids) + ")")
)

victor_orders_ids = [str(row.id) for row in session.execute("""
    select id from orders where client_name='Victor'
""")]
print_result_set(
    session.execute("select avg(price) from order_price where id in (" + ', '.join(victor_orders_ids) + ")")
)

fedya_orders_ids = [str(row.id) for row in session.execute("""
    select id from orders where client_name='Fedya'
""")]
print_result_set(
    session.execute("select avg(price) from order_price where id in (" + ', '.join(fedya_orders_ids) + ")")
)

Row(system_avg_price=316)
Row(system_avg_price=1100)
Row(system_avg_price=1400)


In [128]:
# Для кожного замовників визначте суму на яку були зроблені усі його замовлення
print_result_set(
    session.execute("select sum(price) from order_price where id in (" + ', '.join(ivan_orders_ids) + ")")
)


print_result_set(
    session.execute("select sum(price) from order_price where id in (" + ', '.join(victor_orders_ids) + ")")
)

print_result_set(
    session.execute("select sum(price) from order_price where id in (" + ', '.join(fedya_orders_ids) + ")")
)

Row(system_sum_price=950)
Row(system_sum_price=2200)
Row(system_sum_price=1400)


In [129]:
# Для кожного замовників визначте замовлення з максимальною вартістю
print_result_set(
    session.execute("select max(price) from order_price where id in (" + ', '.join(ivan_orders_ids) + ")")
)


print_result_set(
    session.execute("select max(price) from order_price where id in (" + ', '.join(victor_orders_ids) + ")")
)

print_result_set(
    session.execute("select max(price) from order_price where id in (" + ', '.join(fedya_orders_ids) + ")")
)

Row(system_max_price=600)
Row(system_max_price=2000)
Row(system_max_price=1400)


In [132]:
#  Модифікуйте певне замовлення додавши / видаливши один або кілька товарів при цьому також змінюючи вартість замовлення

# 
#  можна було не використовувати counter табл а діставати значення і модифікувати їх в пайтоні 
# 
print_result_set(
    session.execute("""
             UPDATE orders 
             SET products = products - [1]
             WHERE client_name = 'Ivan' AND date='2020-12-12T12:00:00' AND id = 0;
     """)
 )
print_result_set(
    session.execute("""            
             UPDATE order_price 
             SET price = price - 100
             WHERE id = 0;
     """)
 )

print_result_set(
    session.execute("""
        SELECT * from orders
        WHERE client_name='Ivan' AND date='2020-12-12T12:00:00';
    """)
)
print_result_set(
    session.execute("""
        SELECT * from order_price
        WHERE id = 0;
    """)
)

Row(client_name='Ivan', date=datetime.datetime(2020, 12, 12, 12, 0), id=0, products=[2, 3])
Row(id=0, price=400)


In [135]:
# Для кожного замовлення виведіть час коли його ціна були занесена в базу (SELECT WRITETIME)
print_result_set(
    session.execute("""            
             SELECT WRITETIME (price), id FROM order_price;
     """)
 )


Row(writetime_price=1615731073020707, id=5)
Row(writetime_price=1615731072926329, id=1)
Row(writetime_price=1615732341465632, id=0)
Row(writetime_price=1615731072945888, id=2)
Row(writetime_price=1615731072988952, id=4)
Row(writetime_price=1615731072957215, id=3)


In [136]:
#  Створіть замовлення з певним часом життя (TTL), після якого воно видалиться 
print_result_set(
    session.execute("""            
            INSERT INTO orders (client_name, date, products, id) 
            VALUES ('Kate', '2021-01-12T12:00:00', [55], 6)
            USING TTL 86400;
     """)
 )

In [137]:
# Поверніть замовлення у форматі JSON
print_result_set(
    session.execute("""
        SELECT json * from orders
        WHERE client_name = 'Kate';
    """)
)

Row(json='{"client_name": "Kate", "date": "2021-01-12 12:00:00.000Z", "id": 6, "products": [55]}')


In [140]:
# Додайте замовлення у форматі JSON
print_result_set(
    session.execute("""
        INSERT INTO orders JSON '{
          "client_name" : "Filip", 
          "date": "2021-01-12 12:00:00.000Z", 
          "id" : 7,
          "products": [55, 100] }';
    """)
)
print_result_set(
    session.execute("""
        SELECT * from orders
        WHERE client_name = 'Filip';
    """)
)

Row(client_name='Filip', date=datetime.datetime(2021, 1, 12, 12, 0), id=7, products=[55, 100])
